# CRAFTY

# Clustering and scRAping For biTcoin deanonYmization

## Dataset e gestione delle memoria

Importiamo i dataset con `pandas` cercando di risparmiare memoria andando a specificare,
tramite il parametro `dtype` il tipo e la dimensione dei dati di ogni colonna.

In questo modo risparmiamo quasi 1GB di RAM e velocizziamo il caricamento del dataset. Sono
stato in parte costretto a procedere in questo modo poiché non sempre riuscivo a contenere
tutti i dati generati nella RAM della macchina (8GB di cui metà occupati da processi di
sistema).

Più avanti andrò inoltre a deallocare i DataFrame più grandi una volta che questi non sono
più necessari. Questo potrebbe creare qualche problema nel caso si voglia eseguire tutto il
notebook e in seguito eseguire qualche cella specifica poiché si potrebbe far riferimento a
dati già deallocati.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import networkx as nx
import bs4
import requests
import time

tx_df = pd.read_csv(
    "transactions.csv",
    names=["timestamp", "blk", "tx", "is_coinbase", "fee"],
    dtype={
        "timestamp": np.uint32,
        "blk": np.uint32,
        "tx": np.uint32,
        "is_coinbase": np.uint8,
        "fee": np.uint64,
    },
)
tx_df["timestamp"] = pd.to_datetime(tx_df["timestamp"], unit="s")
tx_df.info()

## Analisi

In questa prima fase andiamo ad effettuare delle analisi generali sul dataset.

### Distribuzione del numero di transazioni per blocco

Vogliamo ricavare la **distribuzione** del numero di transazioni per blocco, nell'intero
periodo temporale considerato.

Per prima cosa raggruppiamo il dataset per blocco.


In [ ]:
blk_grp = tx_df.groupby("blk")

Dato che ogni riga del dataset rappresenta una transazione e dato che ogni transazione ha un
identificatore univoco, questa comparirà al più una volta nel dataset.

Siamo quindi in grado di ottenere il numero di transazioni per blocco tramite la funzione di
aggregazione `count`.


In [ ]:
tx_per_blk = blk_grp["tx"].count()
tx_per_blk

#### Plot

Come sistema di plotting per la distribuzione del numero di transazioni per blocco abbiamo
due punti di vista:

- Istogramma delle frequenze di ogni dimensione dei blocchi trovata.
- Distribuzione della dimensione dei blocchi nel tempo con un grafico a barre approssimato.

Per plottare la seconda distribuzione ho optato per un grafico a barre ma plottare una barra
per ogni blocco (quasi 200000 blocchi) sarebbe stato troppo lungo.

Ho quindi deciso di raggruppare il dataset ulteriormente. Dato che volevo 30 barre, avevo
bisogno di raggruppare ogni $n$ righe. Dove $n$ viene calcolato dividendo la lunghezza del
dataset per 30.

Ho infine calcolato la media di ogni gruppo ottenuto per avere una rappresentazione del
numero di transazioni
per blocco.


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

ax1.hist(tx_per_blk, bins=20, ec="w")
ax1.set_title("Distribuzione delle transazioni per blocco")
ax1.set_xlabel("Dimensione del blocco")
ax1.set_ylabel("Quantità di blocchi")
ax1.set_yscale("log")

bins = 30
group_size = len(tx_per_blk) // bins
tx_per_blk_approx = tx_per_blk.groupby(np.arange(len(tx_per_blk)) // group_size).mean()

ax2.bar(
    tx_per_blk_approx.index,
    tx_per_blk_approx,
    color="tab:orange",
    label="Transazioni",
)
ax2.set_title("Distribuzione delle transazioni per blocco nel tempo")
ax2.set_xlabel("Tempo")
ax2.set_ylabel("Transazioni per blocco")
ax2.legend()

plt.show()

### Occupazione dei blocchi nel tempo

La seconda analisi prevede lo studio di come l'occupazione dei blocchi si è evoluta nel
tempo. In particolare dobbiamo andare a considerare intervalli di due mesi.

Dato che avevamo già raggruppato il dataset per blocchi, ho deciso di considerare come data
di riferimento, la data della transazione meno recente come data di riferimento di ogni
blocco.


In [ ]:
# transazioni per blocco con timestamp
tx_per_blk = pd.concat([tx_per_blk, blk_grp["timestamp"].min()], axis="columns")
tx_per_blk = tx_per_blk.rename(columns={"tx": "tx_count"})

Ho infine utilizzato l'oggetto `Grouper` di Pandas per riuscire ad interagire comodamente
con le date ed effettuare un raggruppamento bimestrale dei blocchi.

Per ogni gruppo ho infine calcolato l'occupazione media dei blocchi.


In [ ]:
tx_per_blk = (
    tx_per_blk.groupby(pd.Grouper(key="timestamp", freq="2MS"))["tx_count"]
    .mean()
    .to_frame()
    .reset_index()
)

tx_per_blk.head()

#### Plot

Come metodologia di plotting ho optato per un semplice plot in grado di fornire una
rappresentazione accurata dell'evoluzione dell'occupazione media dei blocchi ogni
due mesi.


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(
    tx_per_blk["timestamp"],
    tx_per_blk["tx_count"],
    c="tab:red",
    marker="o",
    mec="k",
    label="transazioni medie per blocco",
)

plt.title("Occupazione media dei blocchi ogni 2 mesi")
plt.xlabel("Data")
plt.ylabel("Numero medio di transazioni per blocco")
plt.grid()
plt.legend()
plt.show()

del tx_per_blk

### Totale UTXO

Passiamo ora a calcolare l'ammontare totale degli **UTXO** al momento dell'ultima transazione
registrata nel dataset.

L'idea è quella di calcolare prima tutto l'output che è stato speso e poi sottrarlo  
dall'output totale prodotto.

Per arrivare a questo risultato effettuiamo un'operazione di `merge` tra il dataset degli
**input** e il dataset degli **output**.

Dato che non abbiamo identificatori univoci per ogni output ma solo per le transazioni, per
calcolare l'UTXO totale dobbiamo:

- Considerare le transazioni mai riferite come input da altre transazioni.
- Considerare gli output mai riferiti da altre transazioni che però riferiscono altri output
  all'interno della stessa transazione.

Ho deciso quindi di filtrare il dataset degli output eliminando

Ho deciso quindi di implementare l'operazione di `merge` (in questo caso di tipo _inner_)
basandomi sia sulle transazioni che sugli output riferiti in input.

Il risultato sarà un DataFrame `spent_df` in cui ogni riga è identificata univocamente dalla
coppia `(in_tx, out_pos)`.

In questo modo otterremo un dataset in cui saranno presenti solo le transazioni riferite
come input da altre transazioni.


In [ ]:
in_df = pd.read_csv(
    "inputs.csv",
    names=["tx", "utxo_tx", "utxo_out_pos"],
    dtype={"tx": np.uint32, "utxo_tx": np.uint32, "out_pos": np.uint16},
)

out_df = pd.read_csv(
    "outputs.csv",
    names=["tx", "out_pos", "address", "amount", "script_type"],
    dtype={
        "tx": np.uint32,
        "out_pos": np.uint16,
        "address": np.uint32,
        "amount": np.uint64,
        "script_type": np.uint8,
    },
)

spent_df = (
    out_df.merge(
        in_df,
        how="inner",
        left_on=["tx", "out_pos"],
        right_on=["utxo_tx", "utxo_out_pos"],
    )
    .rename(columns={"tx_x": "in_tx", "tx_y": "tx"})
    .drop(columns=["utxo_tx", "utxo_out_pos", "script_type"])
)

del in_df

print(spent_df.info())
spent_df

A questo punto possiamo sommare i valori della colonna `amount` del DataFrame `spent_df`
per ricavare l'output speso.

Se invece sommiamo i valori della colonna `amount` del dataset degli **output** generati in
tutto il periodo di tempo considerato ricaviamo il valore totale degli output sia speso che
non speso.

Calcolando la differenza tra il il valore totale degli output e il valore totale degli
output spesi ricaviamo l'**UTXO** totale.


In [ ]:
utxo = out_df["amount"].sum() - spent_df["amount"].sum()
print(f"UTXO totale: {utxo}")

del out_df

#### Plot

Tramite un grafico a torta è possibile comparare l'output speso con l'**UTXO**.


In [ ]:
plt.figure(figsize=(10, 7))
plt.pie(
    [utxo, spent_df["amount"].sum()],
    explode=(0.1, 0),
    labels=["UTXO", "Totale speso"],
    colors=["tab:green", "tab:red"],
    autopct="%.2f%%",
)

plt.title("UTXO")
plt.legend()
plt.show()

### Distribuzione intervalli di tempo per il consumo di un output

Abbiamo già a disposizione un DataFrame che mette in correlazione gli input e gli output.

Per capire dopo quanto tempo è stato speso un output dobbiamo aggiungere il valore di
`timestamp` sia per il valore in input che per il valore in output.

Per riuscire a farlo ho effettuato un doppio `merge` con il DataFrame delle transazioni.


In [ ]:
spent_time_df = (
    spent_df.merge(tx_df, how="inner", on="tx")
    .drop(columns=["blk", "tx", "is_coinbase", "fee"])
    .rename(columns={"timestamp": "spent_time"})
)

spent_time_df = (
    spent_time_df.merge(tx_df, how="inner", left_on="in_tx", right_on="tx")
    .drop(columns=["blk", "tx", "is_coinbase", "fee"])
    .rename(columns={"timestamp": "gen_time"})
)

print(spent_df.info())
spent_df

Per ottenere gli intervalli possiamo effettuare la differenza tra il momento della creazione
di un output e il momento in cui questo viene speso, ossia quando viene riferito come input
da un transazione successiva.


In [ ]:
days = (spent_time_df["spent_time"] - spent_time_df["gen_time"]).dt.days

#### Plot

A questo punto abbiamo una Series `days`, contenente il numero di giorni di vita di ogni
transazione.

Sull'asse $x$ abbiamo la lunghezza degli intervalli di tempo (in giorni) prima che un output
venisse speso.

Sull'asse $y$ abbiamo invece la quantità di output che vengono spesi ogni $x$ giorni.


In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(days, bins=40, ec="w", color="teal")

plt.title("Distribuzione degli intervalli di tempo")
plt.xlabel("Giorni prima di spendere l'output")
plt.ylabel("Quantità di output")
plt.yscale("log")

### Distribuzione delle fee

In quest'ultima fase vogliamo effettuare due analisi:

1. Mettere a confronto la distribuzione di _fee_ nulle e la distribuzione di _fee_ non nulle
   nel tempo.
2. Vedere se esiste una correlazione tra il numero e il valor medio mensile di _fee_ non
   nulle.

Dato che le transazioni coinbase non hanno mai _fee_ non le consideriamo in quanto siamo
interessati solo alle transazioni che potrebbero potenzialmente avere _fee_ non nulle.

#### Distribuzione delle _fee_ nulle e non nulle a confronto

In questa prima fase vogliamo vedere come il numero di fee nulle e non nulle è cambiato mese
per mese.


In [ ]:
tx_df = tx_df[tx_df["is_coinbase"] == 0]

zero_fee = tx_df[tx_df["fee"] == 0]
non_zero_fee = tx_df[tx_df["fee"] > 0]

zero_count = (
    zero_fee.groupby(pd.Grouper(key="timestamp", freq="1MS"))["fee"]
    .count()
    .reset_index()
)

non_zero_count = (
    non_zero_fee.groupby(pd.Grouper(key="timestamp", freq="1MS"))["fee"]
    .count()
    .reset_index()
    .dropna()
)

del tx_df

##### Plot

Il tipo di grafico è un grouped bar plot.


In [ ]:
interval = datetime.timedelta(days=10)
fig, ax = plt.subplots(1, 1, figsize=(15, 6))
ax.bar(
    pd.to_datetime(zero_count["timestamp"]) - interval / 2,
    zero_count["fee"],
    width=interval,
    label="Fee nulle",
)

ax.bar(
    non_zero_count["timestamp"] + interval / 2,
    non_zero_count["fee"],
    width=interval,
    label="Fee non nulle",
)

ax.set_title("Fee nulle e non nulle a confronto")
ax.set_xlabel("Mese")
ax.set_ylabel("Quantità di fee")
fig.autofmt_xdate()
ax.set_yscale("log")
ax.legend()
plt.show()

#### Correlazione tra numero di _fee_ e il loro valore

Vogliamo ora vedere se c'è una qualche correlazione tra il numero mensile di _fee_ non nulle
e il valor medio mensile.


In [ ]:
non_zero_mean_value = (
    non_zero_fee.groupby(pd.Grouper(key="timestamp", freq="1MS"))["fee"]
    .mean()
    .reset_index()
    .dropna()
)

fee_df = non_zero_count.merge(non_zero_mean_value, how="inner", on="timestamp")

print(fee_df.info())
fee_df.head()

#### Plot

Dato che abbiamo calcolato due distribuzioni ho optato per effettuare un doppio plot.


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
fig.autofmt_xdate()

ax.plot(
    fee_df["timestamp"],
    fee_df["fee_x"],
    "o-",
    mec="k",
    label="quantità mensile delle fee",
    c="seagreen",
)

ax.plot(
    fee_df["timestamp"],
    fee_df["fee_y"],
    "o-",
    mec="k",
    label="valor medio mensile delle fee",
    c="tab:orange",
)

ax.set_title("Distribuzione quantità e valor medio delle fee delle nulle")
ax.set_xlabel("Data")
ax.set_yscale("log")
ax.grid()
ax.legend()

plt.show()

del fee_df

Come possiamo vedere dai grafici il numero di _fee_ non nulle tende a crescere nel tempo
mentre il loro valor medio tende a diminuire.

Per riuscire a comprendere meglio questo trend si dovrebbe mettere in correlazione anche il  
numero di transazioni mensili.

Ipotizzo che con il crescere della popolarità di bitcoin, gli utenti abbiano anche iniziato a
partecipare in maniera più attiva.

Ad esempio ricompensando, anche se con delle _fee_ basse, i miner che permettono di creare
nuovi blocchi.


## Clusterizzazione degli indirizzi

In questa fase vogliamo implementare l'**euristica multi-input** per la clusterizzazione
degli indirizzi.

L'obbiettivo è quello di riunire in cluster, gli indirizzi appartenenti ad uno stesso utente.

### Grafo degli indirizzi

Per costruire il grafo degli indirizzi dobbiamo:

1. Individuare le transazioni usate come input di ogni transazione.
2. Individuare gli indirizzi di tali transazioni.
3. Per ogni transazione, collegare il primo indirizzo della prima transazione di input con
   tutti gli altri indirizzi delle altre transazioni di input.

Utilizziamo il DataFrame `spent_df` creato in precedenza per il calcolo dell'UTXO poiché
mette sulla stessa riga due transazioni:

- Sulla colonna `tx` troviamo transazioni che riferiscono altre transazioni come loro input.
- Sulla colonna `in_tx` troviamo transazioni riferite come input e di cui conosciamo
  l'address di destinazione.

L'euristica di **clustering** prevede inoltre che si considerino transazioni con più di un
input, sarà necessario filtrare il dataset di conseguenza.


In [ ]:
mapping_df = pd.read_csv(
    "mappings.csv", names=["hash", "address"], dtype={"hash": str, "address": np.uint32}
)

more_than_1 = spent_df.groupby("tx")["in_tx"].count().reset_index()
more_than_1 = more_than_1[more_than_1["in_tx"] > 1].rename(
    columns={"in_tx": "in_count"}
)
more_than_1 = more_than_1.merge(spent_df, how="inner", on="tx").drop(
    columns=["in_count"]
)

more_than_1 = more_than_1.merge(mapping_df, how="inner", on="address")

del spent_df
del mapping_df

Ora possiamo popolare il grafo andando a prendere per ogni transazione l'indirizzo del primo
input per collegarlo agli indirizzi degli altri input della stessa transazione.

Per evitare archi in loop sullo stesso nodo controlliamo che i due nodi siano differenti
prima di aggiungere l'arco.

Non è necessario controllare che un arco venga aggiunto più volte poiché, se ci provassimo,
il risultato sarebbe in quanto il controllo viene già fatto internamente da `NetworkX`.

Non ho aggiunto tutti gli address presenti nel dataset di mapping come nodi del grafo:

- Per risparmiare memoria.
- Molti sarebbero rimasti nodi singoli se non riferiti.

Calcoliamo infine le componenti connesse ottenendo i cluster e ordiniamoli per dimensione.


In [ ]:
addr_graph = nx.Graph()
addresses = more_than_1.groupby("tx")["hash"].apply(list)
for a in addresses:
    for i in range(1, len(a)):
        if a[0] != a[i]:
            addr_graph.add_edge(a[0], a[i])

clusters = sorted(nx.connected_components(addr_graph), key=len, reverse=True)

del addr_graph
del more_than_1

### Analisi cluster

Procediamo ora con qualche analisi sui cluster ottenuti come

- Dimensione media, minima e massima dei cluster.
- Distribuzione della dimensione dei cluster.

Uno degli obbiettivi è quello di mettere in evidenza la distribuzione _power law_ della
dimensione dei cluster.


In [ ]:
cluster_sizes = [len(cluster) for cluster in clusters]

print(f"Numero di cluster trovati: {len(clusters)}")
print(f"Dimensione media: {np.mean(cluster_sizes):.4f}")
print(f"Dimensione minima: {cluster_sizes[-1]}")
print(f"Dimensione massima: {cluster_sizes[0]}")
print(f"Dimensione dei 10 cluster più grandi: {cluster_sizes[:10]}")

#### Plot

Come plot ho optato per

- Un istogramma delle frequenze per ciascuna dimensione
- Uno scatter plot su scala logaritmica per evidenziare come la dimensione dei cluster
  segua una lo stesso pattern di una distribuzione power law, tipica delle reti scale
  free.


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

ax1.hist(cluster_sizes, ec="w", color="darkred")
ax1.set_title("Distribuzione della dimensione dei cluster")
ax1.set_xlabel("Dimensione cluster")
ax1.set_ylabel("Frequenza della dimensione")
ax1.set_yscale("log")

cluster_sizes_count = {cs: 0 for cs in cluster_sizes}
for cs in cluster_sizes:
    cluster_sizes_count[cs] += 1

data = list(zip(*[(k, cluster_sizes_count[k]) for k in cluster_sizes_count.keys()]))

ax2.scatter(list(data[0]), list(data[1]), marker="x", c="chocolate")
ax2.set_xlabel("Dimensione cluster")
ax2.set_ylabel("Frequenza della dimensione")
ax2.set_xscale("log")
ax2.set_yscale("log")
ax2.grid()

plt.show()

## Deanonimizzazione

Passiamo alla parte di deanonimizzazione degli indirizzi considerando solo i 10 cluster più
grandi.

Per entrambi i siti ho utilizzato gli strumenti della libreria `BeautifulSoup` in quanto le
pagine fornivano le informazioni rilevanti nel loro codice `html`.

### WalletExplorer

Ho iniziato dal sito [WalletExplorer](https://www.walletexplorer.com/) che si presenta con
una `form` in cui è possibile inserire l'hash dell'indirizzo che cerchiamo.

![wallet_explorer1_img](images/wallet_explorer1.png)

Il `tag` che cerchiamo è un tag di input con il campo `name` equivalente a `q`. Possiamo
dunque effettuare una richiesta `GET` in cui specifichiamo il parametro `params` tramite un
dizionario in modo da ottenere una pagina simile a questa

![wallet_explorer2_img](images/wallet_explorer2.png)

L'informazione che cerchiamo si trova nel primo tag `h2` che troviamo nella pagina.

![wallet_explorer4](images/wallet_explorer4.png)

Una volta agganciato il tag con `BeautifulSoup` possiamo direttamente ricavare il testo
contenuto tramite la funzione `get_text()`.

Se la stringa che ricaviamo è un numero racchiuso tramite parentesi quadre significa che non
siamo riusciti a deanonimizzare l'indirizzo.

Se invece troviamo una parola siamo riusciti a deanonimizzare l'indirizzo e possiamo passare
al cluster successivo.

#### AntiScraping

Nel caso in cui si effetuino troppe richieste in poco tempo alla pagina questa ci blocca
ritornando un codice di errore `429`.

Per evitare di essere bloccato ho limitato a 10 il numero di richieste per ogni cluster e ho
aggiunto un ritardo di 3 secondi tra una richiesta e l'altra.


In [ ]:
base_url = "https://www.walletexplorer.com/"
session = requests.Session()
session.headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    + " (KHTML, like Gecko) Chrome/61.0.3163.100Safari/537.36"
}


def scrape_wallet_explorer(address: str) -> None | str:
    r = session.get(base_url, params={"q": address})
    if r.status_code == 200:
        soup = bs4.BeautifulSoup(r.text, "html.parser")
        wallet_tag = soup.find("h2")
        if isinstance(wallet_tag, bs4.Tag):
            text = wallet_tag.get_text()
            wallet_name = text.split(" ")[1]
            if wallet_name.startswith("["):
                return None
            else:
                return wallet_name
    else:
        return f"Errore {r.status_code}: {r.reason}"


too_many_requests = False
for cluster_ID, cluster in enumerate(clusters[:10], start=1):
    if too_many_requests == True:
        break
    for addr_ID, addr in enumerate(cluster):
        if addr_ID < 10:
            time.sleep(3)
            wallet_name = scrape_wallet_explorer(addr)
            if wallet_name != None:
                if wallet_name.startswith("Errore"):
                    too_many_requests = True
                print(f"Cluster {cluster_ID}: {wallet_name}")
                break
        else:
            print(f"Cluster {cluster_ID}: Anonimo")
            break

session.close()

### BitInfoCharts

Per la seconda fase di scraping esploriamo il sito [BitInfoCharts]
(https://bitinfocharts.com/) la cui URL può essere estesa con

`/bitcoin/address/<address hash>`

per ottenere una pagina di questo tipo

![bit_info_charts1](images/bit_info_charts2.png)

Come vediamo all'interno della pagina è presente un tag di tipo `small` in cui troviamo
il nome di un _wallet_.

Se il nome del wallet non è composto da sole cifre, allora siamo riusciti a deanonimizzare
l'indirizzo, in caso contrario dobbiamo compiere altri tentativi.

#### Antiscraping

Questo sito permette maggiore libertà in fase di scraping, infatti sono riuscito ad inviare
anche centinaia di richieste per tentare di deanonimizzare gli indirizzi di uno stesso
cluster. Il tutto senza aggiungere ritardo tra l'una e l'altra richiesta.

In ogni caso dopo un troppi tentativi si viene bloccati e il blocco può durare anche giorni.
Trovo quindi necessario limitare il numero di richieste per poter effettuare più prove.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


driver = webdriver.Chrome()
base_url = "https://bitinfocharts.com/bitcoin/address/"


def scrape_bit_info_charts(address: str) -> None | str:
    try:
        driver.get(base_url + address)
        # small_tags = WebDriverWait(driver, 10).until(
        #     EC.presence_of_all_elements_located((By.TAG_NAME, "small"))
        # )
        small_tags = driver.find_elements(By.TAG_NAME, "small")
        for tag in small_tags:
            if tag.text.startswith("wallet:"):
                return tag.text.split(" ")[1]
    except TimeoutException:
        return None


http_error = False
for cluster_ID, cluster in enumerate(clusters[:10], start=1):
    if http_error == True:
        break
    for addr_ID, addr in enumerate(cluster):
        if addr_ID < 10:
            # time.sleep(2)
            wallet_name = scrape_bit_info_charts(addr)
            if wallet_name != None and not wallet_name.isdigit():
                if wallet_name.startswith("Errore"):
                    http_error = True
                    print(wallet_name)
                    break
                print(f"Cluster {cluster_ID}: {wallet_name}")
                break
        else:
            print(f"Cluster {cluster_ID}: Anonimo")
            break

driver.quit()

In [ ]:
base_url = "https://bitinfocharts.com/bitcoin/address/"
session = requests.Session()
session.headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    + " (KHTML, like Gecko) Chrome/61.0.3163.100Safari/537.36"
}


def scrape_bit_info_charts(address: str) -> None | str:
    r = session.get(base_url + address)
    if r.status_code == 200:
        soup = bs4.BeautifulSoup(r.text, "html.parser")
        small_tags = soup.find_all("small")
        for tag in small_tags:
            if isinstance(tag, bs4.Tag):
                text = tag.get_text()
                if text.startswith("wallet:"):
                    return text.split(" ")[1]

        return None
    else:
        return f"Errore {r.status_code}: {r.reason}"


http_error = False
for cluster_ID, cluster in enumerate(clusters[:10], start=1):
    if http_error == True:
        break
    for addr_ID, addr in enumerate(cluster):
        if addr_ID < 10:
            # time.sleep(2)
            wallet_name = scrape_bit_info_charts(addr)
            if wallet_name != None and not wallet_name.isdigit():
                if wallet_name.startswith("Errore"):
                    http_error = True
                    print(wallet_name)
                    break
                print(f"Cluster {cluster_ID}: {wallet_name}")
                break
        else:
            print(f"Cluster {cluster_ID}: Anonimo")
            break

session.close()

Per entrambe le pagine ho utilizzato la libreria `requests` unita a `B`


### Conclusioni

Tramite il sito [WalletExplorer](https://www.walletexplorer.com/) sono riuscito a
deanonimizzare 5 indirizzi.

- **CoinJoinMess**: servizio per combinare più pagamenti in una singola transazione per
  rendere più difficile la deanonimizzazione degli indirizzi.
- **SilkRoadMarketplace**: mercato illegale, per lo più di merci di contrabbando
- **Instawallet.org**: servizio per effettuare transazioni e garantire un certo grado di
  anonimato.
- **BTC-e.com-old**: un exchanger.
- **BtcDice.com**: servizio di scommesse.

Tramite il sito [BitInfoCharts](https://bitinfocharts.com/) sono invece riuscito a
deanonimizzare 4 servizi.

- **F2Pool**: mining pool.
- **SilkRoadMarketplace**: mercato illegale, per lo più di merci di contrabbando.
- **Instawallet.org**: servizio per effettuare transazioni e garantire un certo grado di
  anonimato.
- **Eligius**: mining pool.

Il fatto che alcuni cluster vengano deanonimizzati in modo differente dai due siti potrebbe
essere imputabile a diversi fattori.

I dati potrebbero non essere aggiornati allo stesso modo dai due siti. Potrebbero inoltre
essere stati adottati algoritmi ed euristiche differenti i quali potrebbero differire in
precisione e dunque portare ad errori di vario genere in alcuni casi.
